## Custom Segmentation Algorithm Dev

Notebook to build segmentation implementation ([spheroid_cytometer.py](spheroid_cytometer.py)).

In [1]:
import os
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from scipy import ndimage
from skimage import feature
from scipy import ndimage as ndi
from skimage import morphology
from skimage import filters
from skimage import img_as_float32
from cytokit.ops import cytometry as cytometer_op
from cytokit.ops import tile_generator, tile_crop
from cytokit import math as ck_math
from cytokit import config as ck_config
from itkwidgets import view
%matplotlib inline
%run spheroid_cytometer.py
%run scripts/logging.py

In [2]:
exp_name = '20190215-mc38-dmso-control'
#exp_dir, tile_index = 'XY07', 0
#exp_dir, tile_index = 'XY02', 0 # Example of ambiguous clump 
#exp_dir, tile_index = 'XY02', 4 # Very large clump
#exp_dir, tile_index = 'XY02', 1
exp_dir, tile_index = 'XY03', 2 # 4 clumps together
# exp_dir, tile_index = 'XY03', 5 # A great example of a cell and two clumps all at different elevations
#exp_dir, tile_index = 'XY03', 4 # 3 somewhat close spheroids
data_dir = osp.join(os.environ['CYTOKIT_DATA_DIR'], 'cytokit', 'mc38-spheroid', exp_name, exp_dir, 'raw')
config_dir = osp.join(os.environ['CYTOKIT_REPO_DIR'], 'pub', 'config', 'mc38-spheroid')
config = ck_config.load(config_dir)
os.environ['CYTOKIT_PATH_FORMATS'] = 'get_default_path_formats("1_' + exp_dir + '_{tile:05d}_Z{z:03d}_CH{channel:d}.tif")'
config.register_environment()

In [3]:
tile_gen = tile_generator.CytokitTileGenerator(config, data_dir, 0, tile_index)
tile_cropper = tile_crop.CytokitTileCrop(config)

In [4]:
tile = tile_cropper.run(tile_gen.run())
#tile = tile_gen.run()
tile.shape, tile.dtype

((1, 26, 3, 1008, 1344), dtype('uint16'))

In [5]:
from skimage import transform
img_lg = tile[0, :, 0]
img = transform.rescale(img_lg, (1, .5, .5), multichannel=False, preserve_range=True, anti_aliasing=True, mode='reflect').astype(img_lg.dtype)
img.shape, img.dtype

((26, 504, 672), dtype('uint16'))

### Debugging

In [51]:
#view(img)

In [12]:

# def flat_ball(r, rz):
#     return np.stack([
#         np.pad(morphology.disk(rz), r-rz, 'constant'),
#         morphology.disk(r),
#         np.pad(morphology.disk(rz), r-rz, 'constant'),
#     ]).astype(bool)

# def flat_cross(r, rz):
#     return np.stack([
#         np.pad(morphology.disk(rz), r-rz, 'constant'),
#         morphology.disk(r),
#         np.pad(morphology.disk(rz), r-rz, 'constant'),
#     ]).astype(bool)

# def prep(img):
#     img = ndimage.median_filter(img, size=(1, 3, 3))
#     img = img_as_float(img)
#     return img

# def segment(img):
#     # Run small XY median filter to remove outliers (and convert to float)
#     img = ndi.median_filter(img, size=(1, 3, 3))
#     img = img_as_float(img)

#     scalings = np.array([1./26.494136847515644, 1.0, 1.0])
#     img1 = img - ndi.gaussian_filter(img, sigma=scalings*3, mode='reflect')
#     img2 = ndi.grey_opening(img1, structure=flat_ball(15, 4))
#     return img2
#img_res = segment(img)

In [14]:
#view(img_res.astype(np.float32))

In [28]:
%run spheroid_cytometer.py

zfac = 26.4941368475
#ctm = SpheroidCytometer(config=None, sampling=(zfac, 1, 1))
ctm = SpheroidCytometer(config=config)
img_seg = ctm.segment(img_lg, include_intermediate_results=True, 
    rescaling=.5, sigmas=(1, 3, 2), min_peak_dist=75, basin_type='dist')

2019-02-25 11:11:28,813:DEBUG:__main__: Cytometer initialized with sampling rates (26.494136847515644, 1.0, 1.0)
2019-02-25 11:11:28,815:DEBUG:__main__: Rescaling sigma values with sampling factors = (0.0377442, 1.0, 1.0), resize factors = (1, 0.5, 0.5)
2019-02-25 11:11:28,817:DEBUG:__main__: Reducing image XY dimensions by factor (1, 0.5, 0.5)
2019-02-25 11:11:32,101:DEBUG:__main__: Running high pass filter (sigma = (0.0377442, 0.5, 0.5))
2019-02-25 11:11:32,426:DEBUG:__main__: Computing gradients (sigma = (0.11323259999999999, 1.5, 1.5))
2019-02-25 11:11:33,649:DEBUG:__main__: Computing gradient mask (threshold = 0.142578125)
2019-02-25 11:11:33,659:DEBUG:__main__: Applying morphological smoothings to mask
2019-02-25 11:11:39,983:DEBUG:__main__: Computing mask distance transform (sigma = (0.0754884, 1.0, 1.0), sampling = (26.494136847515644, 1.0, 1.0))
2019-02-25 11:11:42,244:DEBUG:__main__: Finding local maxima (min peak dist = 75, max peaks = 75)
2019-02-25 11:11:43,965:DEBUG:__mai

In [26]:
# img_grad = img_seg[:, 4].copy().astype(np.float32)
# img_grad = ndi.gaussian_filter(img_grad, sigma=np.array([1/26.494136847515644, 1., 1.])*24)
# img_grad = (img_grad > filters.threshold_otsu(img_grad)).astype(np.float32)
# view(img_grad)

In [29]:
img0 = exposure.rescale_intensity(img_as_float(img_lg), out_range='float')
img1 = exposure.rescale_intensity(img_as_float(img_seg[:, 2]), out_range='float')
view(
    (img0 + np.where(img1 > 0, 1.2 + .8*img1, 0)).astype(np.float32),
    size_limit_3d=[1024, 1024, 1024]
)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

In [ ]:
feature.peak_local_max()

In [8]:
# Basin
view(
    img_seg[:, -3].astype(np.float32).copy(),
    size_limit_3d=[1024, 1024, 1024]
)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

In [348]:
view(
    img_seg[:, 0].astype(np.float32).copy(),
    size_limit_3d=[1024, 1024, 1024]
)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

In [9]:
df = ctm.quantify(tile, img_seg)
df.info()

2019-02-24 12:43:43,714:DEBUG:__main__: Computing LoG image on channel "BF" (sigma = (0.0377442, 1.0, 1.0))
2019-02-24 12:43:47,689:DEBUG:__main__: Appending LoG image and running quantification


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 18 columns):
id                48 non-null int64
x                 48 non-null float64
y                 48 non-null float64
z                 48 non-null int64
cm:size           48 non-null int64
cm:diameter       48 non-null float64
cm:perimeter      48 non-null float64
cm:solidity       48 non-null float64
cm:circularity    48 non-null float64
nm:size           48 non-null int64
nm:diameter       48 non-null float64
nm:perimeter      48 non-null float64
nm:solidity       48 non-null float64
nm:circularity    48 non-null float64
ci:000:mean       48 non-null float64
ci:001:mean       48 non-null float64
ci:002:mean       48 non-null float64
ci:003:mean       48 non-null float64
dtypes: float64(14), int64(4)
memory usage: 6.8 KB


In [9]:
# plt.imshow((img_bound.astype(np.float32) * .5 + img_as_float(img).astype(np.float32))[13])
# plt.gcf().set_size_inches(24, 24)

### Pipeline Dev

In [10]:
op = cytometer_op.Cytometry2D(config)

In [11]:
op.initialize()

2019-02-24 12:44:44,883:DEBUG:spheroid_cytometer: Cytometer initialized with sampling rates (26.494136847515644, 1.0, 1.0)


In [7]:
# %%time
# img_seg = op.cytometer.segment(tile[0, :, 0], rescale_factor=None, min_object_size=1024, min_peak_dist=200)
# img_seg.shape

In [8]:
%%time
img_seg = op.cytometer.segment(tile[0, :, 0], rescale_factor=.5, min_object_size=512, min_peak_dist=75)
img_seg.shape

CPU times: user 1min 6s, sys: 11.7 s, total: 1min 17s
Wall time: 1min 17s


In [9]:
#view(img_seg[:, 5].astype(np.float32).copy())

In [10]:
%%time
stats = op.cytometer.quantify(tile, img_seg, sigma=(.1, 1, 1), channel_names=config.channel_names, 
                              cell_graph=True, morphology_features=False, 
                              cell_intensity=['mean', 'sum'])

CPU times: user 4min 4s, sys: 3.28 s, total: 4min 7s
Wall time: 4min 7s


In [11]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 26 columns):
id                     371 non-null int64
x                      371 non-null float64
y                      371 non-null float64
z                      371 non-null int64
cm:size                371 non-null int64
cm:diameter            371 non-null float64
cm:perimeter           371 non-null float64
cm:solidity            371 non-null float64
cm:circularity         371 non-null float64
nm:size                371 non-null int64
nm:diameter            371 non-null float64
nm:perimeter           371 non-null float64
nm:solidity            371 non-null float64
nm:circularity         371 non-null float64
cg:n_neighbors         371 non-null int64
cg:neighbor_ids        371 non-null object
cg:adj_neighbor_pct    371 non-null object
cg:adj_bg_pct          371 non-null object
ci:BF:mean             371 non-null float64
ci:LIVE:mean           371 non-null float64
ci:DEAD:mean           371 n

In [12]:
stats['id'].nunique()

23